In [ ]:
## 필요한 라이브러리 로딩
import pandas as pd
from datetime import datetime, timedelta
from pandas.api.types import CategoricalDtype
pd.set_option('mode.chained_assignment',  None)

## 1. covid19 원본 데이터셋 로딩
## covid19 데이터 로딩(파일을 다운로드 받은 경우)
df_covid19 = pd.read_csv("D:/R/data/Rnpy//owid-covid-data.csv")

## covid19 데이터 로딩(온라인에서 바로 로딩할 경우)
##df_covid19 = pd.read_csv(“https://covid.ourworldindata.org/data/owid-covid-data.csv”)
## 2. 전체 데이터셋 중 최근 100일간의 데이터를 필터링한 df_covid19_100 생성
##df_covid19['date']를 datetime으로 변환
df_covid19['date'] = pd.to_datetime(df_covid19['date'], format="%Y-%m-%d")

## 대륙 데이터와 최종 데이터로부터 100일 전 데이터 필터링
df_covid19_100 = df_covid19[(df_covid19['iso_code'].isin(['KOR', 'OWID_ASI', 'OWID_EUR', 
'OWID_OCE', 'OWID_NAM', 'OWID_SAM', 'OWID_AFR'])) 
& (df_covid19['date'] >= (max(df_covid19['date']) - timedelta(days = 100)))]

## 대륙명을 한글로 변환
df_covid19_100.loc[df_covid19_100['location'] == 'South Korea', "location"] = '한국'
df_covid19_100.loc[df_covid19_100['location'] == 'Asia', "location"] = '아시아'
df_covid19_100.loc[df_covid19_100['location'] == 'Europe', "location"] = '유럽'
df_covid19_100.loc[df_covid19_100['location'] == 'Oceania', "location"] = '오세아니아'
df_covid19_100.loc[df_covid19_100['location'] == 'North America', "location"] = '북미'
df_covid19_100.loc[df_covid19_100['location'] == 'South America', "location"] = '남미'
df_covid19_100.loc[df_covid19_100['location'] == 'Africa', "location"] = '아프리카'

## 이산형 변수 설정
ord = CategoricalDtype(categories = ['한국', '아시아', '유럽', '북미', '남미', 
'아프리카','오세아니아'], ordered = True)

df_covid19_100['location'] = df_covid19_100['location'].astype(ord)

## date로 정렬
df_covid19_100 = df_covid19_100.sort_values(by = 'date')

## 3. df_covid19_100을 한국과 각 대륙별 열로 배치한 넓은 형태의 데이터프레임으로 변환
df_covid19_100_wide = df_covid19_100.loc[:,['date', 'location', 'new_cases', 
'people_fully_vaccinated_per_hundred']].rename(columns={'new_cases':'확진자', 
'people_fully_vaccinated_per_hundred':'백신접종완료자'})

df_covid19_100_wide = df_covid19_100_wide.pivot(index='date', columns='location', 
values=['확진자', '백신접종완료자']).sort_values(by = 'date')

df_covid19_100_wide.columns = ['확진자_한국', '확진자_아시아', '확진자_유럽', '확진자_북미',  
'확진자_남미', '확진자_아프리카', '확진자_오세아니아', '백신접종완료자_한국', 
'백신접종완료자_아시아', '백신접종완료자_유럽', '백신접종완료자_북미', 
'백신접종완료자_남미', '백신접종완료자_아프리카', '백신접종완료자_오세아니아']

## 4. covid19 데이터를 국가별로 요약한 df_covid19_stat 생성
df_covid19_stat = df_covid19.groupby(['iso_code', 'continent', 'location'], dropna=True).agg(
    인구수 = ('population', 'max'), 
    전체사망자수 = ('new_deaths', 'sum'), 
    백신접종자완료자수 = ('people_fully_vaccinated', 'max'), 
    인구백명당백신접종완료율 = ('people_fully_vaccinated_per_hundred', 'max'), 
    인구백명당부스터접종자수 = ('total_boosters_per_hundred', 'max')).reset_index()

df_covid19_stat['십만명당사망자수'] = round(df_covid19_stat['전체사망자수'] / 
                                    df_covid19_stat['인구수'] * 100000, 5)

df_covid19_stat['백신접종완료율'] = df_covid19_stat['백신접종자완료자수'] / \
df_covid19_stat['인구수']

## 여백 설정을 위한 변수 설정
margins_P = {'t' : 50, 'b' : 25, 'l' : 25, 'r' : 25}

In [ ]:
import dash
from dash import Dash, html, dcc, callback, Output, Input

In [ ]:
###############################################################################################
## APPENDIX A R과 파이썬을 사용한 대시보드 만들기
###############################################################################################

In [ ]:
app = dash.Dash(__name__)

In [ ]:
app.layout = html.Div(children=[
    html.H1(children='Hello Dash')])

In [ ]:
@app.callback(
    Output(component_id='my-output', component_property='children'),
    Input(component_id='my-input', component_property='value')
)
def update_output_div(input_value):
    return f'Output: {input_value}'

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

In [ ]:
## 파이썬에서 Plotly 라이브러리 모듈 로딩
import plotly.graph_objects as go
## dash 앱의 구성을 위한 라이브러리 로딩
import dash
from dash import dcc, html, dcc, callback, Output, Input

## dash 앱의 구성을 위한 데이터 전처리
total_deaths_5_nations_by_day = df_covid19.copy()
total_deaths_5_nations_by_day = total_deaths_5_nations_by_day[
    (total_deaths_5_nations_by_day['iso_code'].isin(
        ['KOR', 'USA', 'JPN', 'GBR', 'FRA']))].dropna(
    subset = ['total_deaths_per_million'])

nations = {'France':'0', 'Japan':'1', 'South Korea':'2', 
           'United Kingdom':'3', 'United States':'4'}

## dash 앱에 들어갈 Plotly 선그래프 생성
fig = go.Figure()

for location, group in total_deaths_5_nations_by_day.groupby('location'):
    fig.add_trace(go.Scatter(
            mode = 'lines', x = group['date'], 
            y = group['total_deaths_per_million'],
            line = dict(dash = nations[location]),
            name = location, connectgaps = True, showlegend = False
        ))

    fig.add_trace(go.Scatter(
        mode = 'text',
        x = group.loc[group['date'] == group['date'].max(), 'date'],
        y = group.loc[group['date'] == group['date'].max(), 'total_deaths_per_million'],
        text = group.loc[group['date'] == group['date'].max(), 'location'],
        showlegend = False, textposition = 'middle right'
    ))

fig.update_layout(title = dict(text = '코로나19 사망자수 추세', x = 0.5),
                  xaxis = dict(title = '',
                               range = [total_deaths_5_nations_by_day['date'].min(),
                                        total_deaths_5_nations_by_day['date'].max()  
                                        + timedelta(days=150)]),
                  yaxis = dict(title = '10만명당 사망자수 누계'))

## indicator 트레이스에서 사용할 각각의 국가별 최대 사망자 추출
deaths_per_million_in_lateast = total_deaths_5_nations_by_day[
    total_deaths_5_nations_by_day['new_deaths_per_million'].isna() == False]

## indicator 트레이스에서 사용할 십만명당 사망자수 추출
deaths_per_million_in_lateast = pd.merge(
    deaths_per_million_in_lateast.groupby('location')['date'].max(), 
    deaths_per_million_in_lateast, 
    on = ("location", 'date'))[['iso_code', 'location', 'date', 'new_deaths_per_million']]

## indicator 트레이스에서 사용할 데이터 전처리
df_gauge = pd.merge(deaths_per_million_in_lateast, 
                    total_deaths_5_nations_by_day.groupby('location')\
                    ['new_deaths_per_million'].max().reset_index(), 
                    on = 'location').sort_values('location')

df_gauge.columns = ('iso_code', 'location', 'date', '최근사망자', '최대사망자')

## 한국 게이지 인디케이터 생성
fig_gauge_kor = go.Figure()

fig_gauge_kor.add_trace(go.Indicator(
    type = 'indicator', mode = "gauge+number", title = df_gauge.iloc[2, 1],
    value = df_gauge.iloc[2, 3], 
    gauge = dict(axis = dict(
        range = (0, df_gauge.iloc[2, 4]*1.2)),
                 steps = [
                     dict(range = (0, (df_gauge.iloc[2, 4])*1.2*0.5), color = "lightgray"),
                     dict(range = ((df_gauge.iloc[2, 4])*1.2*0.5, 
                                   (df_gauge.iloc[2, 4])*1.2*0.75), color = "darkgray"),
                     dict(range = ((df_gauge.iloc[2, 4])*1.2*0.75, (df_gauge.iloc[2, 4])*1.2),
                          color = "gray")],
                 threshold = dict(
                     line = dict(color = 'white'), value = df_gauge.iloc[2, 4]),
                 bar = dict(color = "darkblue")),
    number = dict(suffix = '명')))

## 프랑스 게이지 인디케이터 생성
fig_gauge_fra = go.Figure()

fig_gauge_fra.add_trace(go.Indicator(
    type = 'indicator', mode = "gauge+number", title = df_gauge.iloc[0, 1],
    value = df_gauge.iloc[0, 3],
    gauge = dict(axis = dict(
        range = (0, df_gauge.iloc[0, 4]*1.2)),
                 steps = [
                     dict(range = (0, (df_gauge.iloc[0, 4])*1.2*0.5), color = "lightgray"),
                     dict(range = ((df_gauge.iloc[0, 4])*1.2*0.5, 
                                   (df_gauge.iloc[0, 4])*1.2*0.75), color = "darkgray"),
                     dict(range = ((df_gauge.iloc[0, 4])*1.2*0.75, (df_gauge.iloc[0, 4])*1.2),
                          color = "gray")],
                 threshold = dict(
                     line = dict(color = 'white'), value = df_gauge.iloc[0, 4]), 
                 bar = dict(color = "darkblue")),
    number = dict(suffix = '명')))

## 일본 게이지 인디케이터 생성
fig_gauge_jpn = go.Figure()

fig_gauge_jpn.add_trace(go.Indicator(
    type = 'indicator', mode = "gauge+number", title = df_gauge.iloc[1, 1],
    value = df_gauge.iloc[1, 3],
    gauge = dict(axis = dict(
        range = (0, df_gauge.iloc[1, 4]*1.2)),
                 steps = [
                     dict(range = (0, (df_gauge.iloc[1, 4])*1.2*0.5), color = "lightgray"),
                     dict(range = ((df_gauge.iloc[1, 4])*1.2*0.5, 
                                   (df_gauge.iloc[1, 4])*1.2*0.75), color = "darkgray"),
                     dict(range = ((df_gauge.iloc[1, 4])*1.2*0.75, (df_gauge.iloc[1, 4])*1.2),
                          color = "gray")],
                 threshold = dict(
                     line = dict(color = 'white'), value = df_gauge.iloc[1, 4]),
                 bar = dict(color = "darkblue")),
    number = dict(suffix = '명')))

## 영국 게이지 인디케이터 생성
fig_gauge_gbr = go.Figure()

fig_gauge_gbr.add_trace(go.Indicator(
    type = 'indicator', mode = "gauge+number", title = df_gauge.iloc[3, 1],
    value = df_gauge.iloc[3, 3],
    gauge = dict(axis = dict(
        range = (0, df_gauge.iloc[3, 4]*1.2)),
                 steps = [
                     dict(range = (0, (df_gauge.iloc[3, 4])*1.2*0.5), color = "lightgray"),
                     dict(range = ((df_gauge.iloc[3, 4])*1.2*0.5, 
                                   (df_gauge.iloc[3, 4])*1.2*0.75), color = "darkgray"),
                     dict(range = ((df_gauge.iloc[3, 4])*1.2*0.75, (df_gauge.iloc[3, 4])*1.2),
                          color = "gray")],
                 threshold = dict(
                     line = dict(color = 'white'), value = df_gauge.iloc[3, 4]),
                 bar = dict(color = "darkblue")),
    number = dict(suffix = '명')))

## 미국 게이지 인디케이터 생성
fig_gauge_usa = go.Figure()

fig_gauge_usa.add_trace(go.Indicator(
    type = 'indicator', mode = "gauge+number", title = df_gauge.iloc[4, 1],
    value = df_gauge.iloc[4, 3],
    gauge = dict(axis = dict(
        range = (0, df_gauge.iloc[4, 4]*1.2)),
                 steps = [
                     dict(range = (0, (df_gauge.iloc[4, 4])*1.2*0.5), color = "lightgray"),
                     dict(range = ((df_gauge.iloc[4, 4])*1.2*0.5, 
                                   (df_gauge.iloc[4, 4])*1.2*0.75), color = "darkgray"),
                     dict(range = ((df_gauge.iloc[4, 4])*1.2*0.75, (df_gauge.iloc[4, 4])*1.2),
                          color = "gray")],
                 threshold = dict(
                     line = dict(color = 'white'), value = df_gauge.iloc[4, 4]),
                 bar = dict(color = "darkblue")),
    number = dict(suffix = '명')))

## Dash 앱의 초기화
app = Dash(__name__)

## Dash 앱 레이아웃 설정
app.layout = html.Div(children = [
    ## 첫 번째 div 설정
    html.Div(id = 'header', children = [
        html.H1('코로나19 사망자수 추세', style = dict(
            textAlign = 'center', color = 'darkblue'))],
             ),
    ## 두 번째 div 설정
    html.Div(id = 'second block', children = [
        ## 두 번째 좌측 div 설정
        html.Div(children = [
            html.Label('날짜 선택', style = dict(
                position = 'absolute', top = '10%', left = '5%')),
            dcc.DatePickerSingle(id = 'picker', 
                                 date = total_deaths_5_nations_by_day['date'].max(),
                                 placeholder='Select a date', clearable=True,
                                 style = dict(position = 'absolute', top = '15%', 
                                              left = '3%', width = '10px'))],
                 style = dict(width = '20%', display = 'inline-block')),
        ## 두 번째 우측 div 설정
        html.Div(
            dcc.Graph(id = 'line_5_nations', figure=fig),
            style = dict(width = '80%', display = 'inline-block'))
        ]),
    ## 세 번째 div 설정
    html.Div(id = 'third block', children = [
        dcc.Graph(id = 'indicator_kor',figure = fig_gauge_kor, 
                  style = dict(width = '20%', display = 'inline-block')),
        dcc.Graph(id = 'indicator_fra',figure = fig_gauge_fra, 
                  style = dict(width = '20%', display = 'inline-block')),
        dcc.Graph(id = 'indicator_jpn',figure = fig_gauge_jpn, 
                  style = dict(width = '20%', display = 'inline-block')),
        dcc.Graph(id = 'indicator_gbr',figure = fig_gauge_gbr, 
                  style = dict(width = '20%', display = 'inline-block')),
        dcc.Graph(id = 'indicator_usa',figure = fig_gauge_usa, 
                  style = dict(width = '20%', display = 'inline-block'))
        ]),
    html.Div(id='output-container-date-picker-single')
    ])

################################################
## 콜백 추가 부분
###############################################

app.run_server(debug=True, use_reloader = False)

In [ ]:
@app.callback(
    Output(component_id = 'indicator_kor', component_property = 'figure'),
    Output(component_id = 'indicator_fra', component_property = 'figure'),
    Output(component_id = 'indicator_jpn', component_property = 'figure'),
    Output(component_id = 'indicator_gbr', component_property = 'figure'),
    Output(component_id = 'indicator_usa', component_property = 'figure'),
    Output(component_id = 'line_5_nations', component_property = 'figure'),
    Input(component_id = 'picker', component_property = 'date'))
def update_output(date_value):
    total_deaths_5_nations_update = total_deaths_5_nations_by_day.loc[
        total_deaths_5_nations_by_day['date'] == date_value, 
        ['date', 'iso_code', 'new_deaths_per_million']].sort_values(by = 'iso_code')

    ## 한국 게이지 인디케이터 생성
    fig_gauge_kor = go.Figure()
    fig_gauge_kor.add_trace(go.Indicator(
        type = 'indicator', mode = "gauge+number", title = df_gauge.iloc[2, 1],
        value = total_deaths_5_nations_update.iloc[2, 2],
        gauge = dict(axis = dict(
            range = (0, df_gauge.iloc[2, 4]*1.2)),
                     steps = [
                         dict(range = (0, (df_gauge.iloc[2, 4])*1.2*0.5), 
                              color = "lightgray"),
                         dict(range = ((df_gauge.iloc[2, 4])*1.2*0.5, 
                                       (df_gauge.iloc[2, 4])*1.2*0.75),
                              color = "darkgray"),
                         dict(range = ((df_gauge.iloc[2, 4])*1.2*0.75, 
                                       (df_gauge.iloc[2, 4])*1.2),
                              color = "gray")],
                     threshold = dict(
                         line = dict(color = 'white'), value = df_gauge.iloc[2, 4]),
                     bar = dict(color = "darkblue")),
        number = dict(suffix = '명')))
    
    ## 프랑스 게이지 인디케이터 생성
    fig_gauge_fra = go.Figure()
    fig_gauge_fra.add_trace(go.Indicator(
        type = 'indicator', mode = "gauge+number", title = df_gauge.iloc[0, 1],
        value = total_deaths_5_nations_update.iloc[0, 2],
        gauge = dict(axis = dict(
            range = (0, df_gauge.iloc[0, 4]*1.2)),
                     steps = [
                         dict(range = (0, (df_gauge.iloc[0, 4])*1.2*0.5), 
                              color = "lightgray"),
                         dict(range = ((df_gauge.iloc[0, 4])*1.2*0.5, 
                                       (df_gauge.iloc[0, 4])*1.2*0.75),
                              color = "darkgray"),
                         dict(range = ((df_gauge.iloc[0, 4])*1.2*0.75, 
                                       (df_gauge.iloc[0, 4])*1.2),
                              color = "gray")],
                     threshold = dict(
                         line = dict(color = 'white'), value = df_gauge.iloc[0, 4]),
                     bar = dict(color = "darkblue")),
        number = dict(suffix = '명')))
    
    ## 일본 게이지 인디케이터 생성
    fig_gauge_jpn = go.Figure()
    
    fig_gauge_jpn.add_trace(go.Indicator(
        type = 'indicator', mode = "gauge+number", title = df_gauge.iloc[1, 1],
        value = total_deaths_5_nations_update.iloc[1, 2],
        gauge = dict(axis = dict(
            range = (0, df_gauge.iloc[1, 4]*1.2)),
                     steps = [
                         dict(range = (0, (df_gauge.iloc[1, 4])*1.2*0.5), 
                              color = "lightgray"),
                         dict(range = ((df_gauge.iloc[1, 4])*1.2*0.5, 
                                       (df_gauge.iloc[1, 4])*1.2*0.75),
                              color = "darkgray"),
                         dict(range = ((df_gauge.iloc[1, 4])*1.2*0.75, 
                                       (df_gauge.iloc[1, 4])*1.2), 
                              color = "gray")],
                     threshold = dict(
                         line = dict(color = 'white'), value = df_gauge.iloc[1, 4]),
                     bar = dict(color = "darkblue")),
        number = dict(suffix = '명')))
    
    ## 영국 게이지 인디케이터 생성
    fig_gauge_gbr = go.Figure()
    
    fig_gauge_gbr.add_trace(go.Indicator(
        type = 'indicator', mode = "gauge+number", title = df_gauge.iloc[3, 1],
        value = total_deaths_5_nations_update.iloc[3, 2],
        gauge = dict(axis = dict(
            range = (0, df_gauge.iloc[3, 4]*1.2)),
                     steps = [
                         dict(range = (0, (df_gauge.iloc[3, 4])*1.2*0.5), 
                              color = "lightgray"),
                         dict(range = ((df_gauge.iloc[3, 4])*1.2*0.5, 
                                       (df_gauge.iloc[3, 4])*1.2*0.75),
                              color = "darkgray"),
                         dict(range = ((df_gauge.iloc[3, 4])*1.2*0.75, 
                                       (df_gauge.iloc[3, 4])*1.2),
                              color = "gray")],
                     threshold = dict(
                         line = dict(color = 'white'), value = df_gauge.iloc[3, 4]),
                     bar = dict(color = "darkblue")),
        number = dict(suffix = '명')))
    
    ## 미국 게이지 인디케이터 생성
    fig_gauge_usa = go.Figure()
    
    fig_gauge_usa.add_trace(go.Indicator(
        type = 'indicator', mode = "gauge+number", title = df_gauge.iloc[4, 1],
        value = total_deaths_5_nations_update.iloc[4, 2],
        gauge = dict(axis = dict(
            range = (0, df_gauge.iloc[4, 4]*1.2)),
                     steps = [
                         dict(range = (0, (df_gauge.iloc[4, 4])*1.2*0.5), 
                              color = "lightgray"),
                         dict(range = ((df_gauge.iloc[4, 4])*1.2*0.5, 
                                       (df_gauge.iloc[4, 4])*1.2*0.75),
                              color = "darkgray"),
                         dict(range = ((df_gauge.iloc[4, 4])*1.2*0.75, 
                                       (df_gauge.iloc[4, 4])*1.2),
                              color = "gray")],
                     threshold = dict(
                         line = dict(color = 'white'), value = df_gauge.iloc[4, 4]),
                     bar = dict(color = "darkblue")),
        number = dict(suffix = '명')))
    
    fig_temp = go.Figure(fig)
    fig_temp = fig_temp.add_shape(
        type = 'line', yref = "y", y0 = 0, 
        y1 = total_deaths_5_nations_by_day['total_deaths_per_million'].max(),
        x0 = date_value, x1 = date_value, line = dict(color = 'black', dash="dot"))
    
    return fig_gauge_kor, fig_gauge_fra, fig_gauge_jpn, fig_gauge_gbr, fig_gauge_usa, fig_temp

In [ ]:
###############################################################################################
## APPENDIX B plotly.express
###############################################################################################

In [ ]:
import plotly.express as px
df = px.data.gapminder()
fig = px.scatter(df.query("year==2007"), x="gdpPercap", y="lifeExp",
                size="pop", color="continent", hover_name="country",
                size_max=60)
fig.show()

In [ ]:
import plotly.express as px
df = px.data.tips()
fig = px.scatter(df, x="total_bill", y="tip", color="smoker", facet_col="sex", facet_row="time")
fig.show()

In [ ]:
import plotly.express as px
df = px.data.iris()
fig = px.scatter(df, x="sepal_width", y="sepal_length", color="species", marginal_y="violin",
                marginal_x="box", trendline="ols", template="simple_white")
fig.show()

In [ ]:
import plotly.express as px
df = px.data.wind()
fig = px.scatter_polar(df, r="frequency", theta="direction",
                        color="strength", symbol="strength", size="frequency")
fig.show()

In [ ]:
import plotly.express as px
df = px.data.gapminder().query("continent == 'Oceania'")
fig = px.line(df, x='year', y='lifeExp', color='country', markers=True)
fig.show()

In [ ]:
import plotly.express as px
df = px.data.stocks()
fig = px.line(df, x='date', y="GOOG", range_x=['2018-07-01','2019-06-30'])
fig.show()

In [ ]:
import plotly.express as px
df = px.data.gapminder().query("continent == 'Europe' and year == 2007 and pop > 2.e6")
fig = px.bar(df, y='pop', x='country', text_auto='.2s',
            title="Default: various text sizes, positions and angles")
fig.show()

In [ ]:
import plotly.express as px
df = px.data.tips()
fig = px.histogram(df, x="day", y="total_bill", color="sex",
            title="Receipts by Payer Gender and Day of Week",
            width=600, height=400,
            labels={ # replaces default labels by column name
                "sex": "Payer Gender", "day": "Day of Week", "total_bill": "Receipts"
            },
            category_orders={ # replaces default order by column name
                "day": ["Thur", "Fri", "Sat", "Sun"], "sex": ["Male", "Female"]
                })
fig.show()

In [ ]:
import plotly.express as px
wide_df = px.data.medals_wide()
fig = px.bar(wide_df, x="nation", y=["gold", "silver", "bronze"], title="Wide-Form Input")
fig.show()

In [ ]:
import plotly.express as px
df = px.data.medals_long()
fig = px.bar(df, x="medal", y="count", color="nation",
            pattern_shape="nation", pattern_shape_sequence=[".", "x", "+"])
fig.show()

In [ ]:
import plotly.express as px
df = px.data.tips()
fig = px.violin(df, y="total_bill")
fig.show()

In [ ]:
import plotly.express as px
df = px.data.tips()
fig = px.violin(df, y="tip", x="smoker", color="sex", box=True, points="all",
                hover_data=df.columns)
fig.show()

In [ ]:
import plotly.express as px
df = px.data.tips()
fig = px.histogram(df, x="total_bill", nbins=20)
fig.show()

In [ ]:
import plotly.express as px
df = px.data.tips()
fig = px.histogram(df, x="total_bill", y="tip", histfunc="avg", nbins=8, text_auto=True)
fig.show()

In [ ]:
import plotly.express as px
df = px.data.tips()
fig = px.pie(df, values='tip', names='day')
fig.show()

In [ ]:
import plotly.express as px
df = px.data.tips()
px.pie(df, values='tip', names='day', hole = 0.3)
fig.show()

In [ ]:
import plotly.express as px
df = px.data.tips()
fig = px.treemap(df, path=[px.Constant("all"), 'sex', 'day', 'time'],
                values='total_bill', color='day')
fig.show()

In [ ]:
import plotly.express as px
df = px.data.tips()
fig = px.treemap(df, path=[px.Constant("all"), 'sex', 'day', 'time'],
                values='total_bill', color='time',
                color_discrete_map={'(?)':'lightgrey', 'Lunch':'gold',
                'Dinner':'darkblue'})
fig.show()

In [ ]:
import plotly.express as px
df = px.data.tips()
fig = px.sunburst(df, path=['day', 'time', 'sex'], values='total_bill')
fig.show()

In [ ]:
import plotly.express as px
data = dict(
    number=[39, 27.4, 20.6, 11, 2],
    stage=["Website visit", "Downloads", "Potential customers", "Requested price",
    "invoice sent"])
fig = px.funnel(data, x='number', y='stage')
fig.show()

In [ ]:
import plotly.express as px
fig = px.choropleth(locations=["CA", "TX", "NY"], locationmode="USA-states", color=[1,2,3], scope="usa")
fig.show()